# Exercise 04 : A/B-testing

## Imports

In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
sql_con = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')

## Using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
- time should have the values: after and before
- avg_diff contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward
- only take into account the users that have observations before and after
- we still are not using the lab ’project1’

In [ ]:
query="""

SELECT
    'before' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN 
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) < julianday(test.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    test
INNER JOIN 
    deadlines
ON
    test.labname != 'project1'
    AND test.labname = deadlines.labs
WHERE
    julianday(test.first_commit_ts) > julianday(test.first_view_ts)
"""

test_results = pd.read_sql(query, sql_con)
test_results

In [3]:
query="""
SELECT
    'before' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN 
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) < julianday(control.first_view_ts)


UNION


SELECT
    'after' AS time,
    AVG(CAST((julianday(control.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT)) AS avg_diff
FROM
    control
INNER JOIN 
    deadlines
ON
    control.labname != 'project1'
    AND control.labname = deadlines.labs
WHERE
    julianday(control.first_commit_ts) > julianday(control.first_view_ts)
"""
control_results = pd.read_sql(query, sql_con)
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286


## Close the connection 

In [4]:
sql_con.close()